### Deploying model in a notebook

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role() # need role to deploy an endpoint

In [ ]:
sess = sagemaker.Session() # to interact with other AWS services
from sagemaker import image_uris

training_image = image_uris.retrieve(
    region = sess.boto_region_name, 
    framework = "object-detection", 
    version = "1" #  '1' is the latest
)
print(training_image) # it is a docker container

In [ ]:
model = sagemaker.model.Model(
    image_uri = training_image,
    model_data = '',    # copy S3 URI .....model.taz.gz
    role = role    
)

In [ ]:
endpoint_name = 'Plastic-dection-endpoint-001'

deployment = model.deploy(
    initial_instance_count = 1,  # the more instances, the faster and more expensive
    instance_type = "ml.m4.xlarge", # Don't need GPU
    endpoint_name = endpoint_name
)

### Visualize detections

In [ ]:
import json 
import boto3
runtime = boto3.client(service_name = "runtime.sagemaker")

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg #pyplot


def visualize_detection(img_file_path,   # we want to get inference from
                        detections, # detections geting from model with the input img_file
                        thresh = 0.3
                       ):
    img = mpimg.imread(img_file_path)
    plt.imshow(img)
    height, width = img.shape    
    colors = dict()
    num_detections = 0
    
    for detection in detections:
        (klass, score, x0, y0, x1, y1) = detection # in LST file we set Plastic bag class is 0
                                                   # so if find a plastic bag the class is 0
        if score < thresh:
            continue
            
        num_detections += 1
        
        # Make color for each class
        cls_id = int(klass)
        if cls_id not in colors:
            colors[cls_id] = (random.random(),random.random(),random.random()) #(0.1,0.5,0.4)
            
        xmin = int(x0*width)
        ymin = int(y0*height)
        xmax = int(x1*width)
        ymax = int(y1*height)         
        
        rect = plt.Rectangle(
            (xmin, ymin),
            xmax - xmin, # width
            ymax - ymin, # height
            fill = False,
            edgecolor = colors[cls_id],
            linewidth = 3.5
        )
        
        plt.gca().add_patch(rect)
        plt.gca().text(
            xmin,      # show text from (xmin, ymin-2) position
            ymin - 2,  
            "{:.3f}".format(score),
            bbox = dict(facecolor = colors[cls_id], alpha = 0.5),
            fontsize = 12,
            color = "white"
        )
    print("Number of detections" + str(num_detections))
    plt.show()

In [ ]:
def show_plastic_prediction(img_file_path, 
                            endpoint_name, 
                            thresh = 0.3):
    byte_array = "" # To send an image to an endpoint to ask for a inference
                    # but need a specific type of structure
    with open(img_file_path, "rb" ) as image:
        f = image.read()
        byte_array = bytearray(f)
        
    endpoint_response = runtime.invoke_endpoint(
        EndpointName = endpoint_name, 
        ContentType = "image/jpeg", 
        Body = byte_array)
    
    results = endpoint_response["Body"].read()
    detections = json.loads(results)    
    
    visualize_detection(img_file_path=img_file_path, 
                        detections=detections['prediction'], 
                        thresh=thresh)
    

## Test endpoint with images from validation dataset

In [ ]:
show_plastic_prediction(
    img_file_path = 'copy_path_of_validation_imd'
    endpoint_name, 
    thresh = 0.3)

## Test endpoint with images from internet

In [ ]:
# Free platic bag photo
# Upload image files to sagemaker (create a "InternetImages" folder inside root)

show_plastic_prediction('InternetImages/pexels-timur-weber-9533031.jpg', 
                        endpoint_name)